## Imports

In [229]:
import pandas as pd
import numpy as np
import pyodbc 
import import_ipynb
import functions as f

import sqlalchemy

In [230]:
# read datasets

df_cases = pd.read_parquet('./Processed/WHO-COVID19-CASES.parquet')

df_vaccines = pd.read_parquet('./Processed/WHO-COVID19-VACCINES.parquet')

df_countries = pd.read_parquet('./Processed/COUNTRIES_DETAILED.parquet')

In [197]:
# Find the latest date in df_cases and df_vaccines

max_cases_df_date = df_cases['DateReported'].max().date()

max_vaccines_df_date = df_vaccines['Date'].max().date()

In [198]:
# SQL Server connection variables

server = 'lirkovsrv.database.windows.net'
database = 'lirkovdb'
username = 'lirkov'
password = '{LLirkoff31}'   
driver= '{ODBC Driver 17 for SQL Server}'

In [199]:
# connect to SQL Server

conn =  pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

In [200]:
# upsert to covid.who_cases SQL Server table

# check the latest 'DateReported' date in the covid.who_cases table
cursor.execute('SELECT MAX(DateReported) FROM covid.who_cases')
max_db_date = cursor.fetchone()

if max_db_date[0] < max_cases_df_date:
    df_cases_upsert = df_cases[df_cases['DateReported'].dt.date > max_db_date[0]]
    rows = 0
    for index, row in df_cases_upsert.iterrows():
        cursor.execute("INSERT INTO covid.who_cases (DateReported, Country, IsoCode, WhoRegion, NewCases, CumulativeCases, NewDeaths, CumulativeDeaths) VALUES(?,?,?,?,?,?,?,?)", row.DateReported, row.Country, row.IsoCode, row.WhoRegion, row.NewCases, row.CumulativeCases, row.NewDeaths, row.CumulativeDeaths)
        # commit every 10000 rows
        rows += 1
        if rows == 10000:
            conn.commit()
            print(f'commiting: {rows}')
            rows = 0
    print(f'{len(df_cases_upsert)} rows have been inserted')
else:
    print('Database is up to date')


conn.commit()
cursor.close()

Database is up to date


In [233]:
# upsert to covid.who_vaccines SQL Server table

# check the latest 'Date' date in the covid.who_vaccines table
cursor.execute('SELECT MAX(Date) FROM covid.who_vaccines')
max_db_date = cursor.fetchone()

try:
    if max_db_date[0] < max_vaccines_df_date:
        df_vaccines_upsert = df_vaccines[df_vaccines['DateReported'].dt.date > max_db_date[0]]
        rows = 0
        for index, row in df_vaccines_upsert.iterrows():
            cursor.execute("INSERT INTO covid.who_vaccines (Location, IsoCode, Date, TotalVaccinations, PeopleVaccinated, PeopleFullyVaccinated, TotalBoosters, DailyVaccinationsRaw, DailyVaccinations, TotalVaccinationsPerHundred, PeopleVaccinatedPerHundred, PeopleFullyVaccinatedPerHundred, TotalBoostersPerHundred, DailyVaccinationsPerMillion, DailyPeopleVaccinated, DailyPeopleVaccinatedPerHundred) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
            row.Location, row.IsoCode, row.Date, row.TotalVaccinations, row.PeopleVaccinated, row.PeopleFullyVaccinated, row.TotalBoosters, row.DailyVaccinationsRaw, row.DailyVaccinations, row.TotalVaccinationsPerHundred, row.PeopleVaccinatedPerHundred, row.PeopleFullyVaccinatedPerHundred, row.TotalBoostersPerHundred, row.DailyVaccinationsPerMillion, row.DailyPeopleVaccinated, row.DailyPeopleVaccinatedPerHundred)
            # commit every 10000 rows
            rows += 1
            if rows == 10000:
                conn.commit()
                print(f'commiting: {rows}')
                rows = 0
        print(f'{len(df_vaccines_upsert)} rows have been inserted')
    else:
        print('Database is up to date')
except TypeError:
    for index, row in df_vaccines.iterrows():
        cursor.execute("INSERT INTO covid.who_vaccines (Location, IsoCode, Date, TotalVaccinations, PeopleVaccinated, PeopleFullyVaccinated, TotalBoosters, DailyVaccinationsRaw, DailyVaccinations, TotalVaccinationsPerHundred, PeopleVaccinatedPerHundred, PeopleFullyVaccinatedPerHundred, TotalBoostersPerHundred, DailyVaccinationsPerMillion, DailyPeopleVaccinated, DailyPeopleVaccinatedPerHundred) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
                       row.Location, row.IsoCode, row.Date, row.TotalVaccinations, row.PeopleVaccinated, row.PeopleFullyVaccinated, row.TotalBoosters, row.DailyVaccinationsRaw, row.DailyVaccinations, row.TotalVaccinationsPerHundred, row.PeopleVaccinatedPerHundred, row.PeopleFullyVaccinatedPerHundred, row.TotalBoostersPerHundred, row.DailyVaccinationsPerMillion, row.DailyPeopleVaccinated, row.DailyPeopleVaccinatedPerHundred)

conn.commit()
cursor.close()


In [231]:
df_vaccines = df_vaccines.replace(np.nan, 0)

In [ ]:
# insert countries data


# cursor.execute("INSERT INTO covid.countries (Continent, Location, IsoCode, Population, PopulationDensity, MedianAge, GdpPerCapita, HumanDevelopmentIndex, LifeExpectancy, Aged65Older, Aged70Older, Alpha2) VALUES(?,?,?,?,?,?,?,?,?,?,?,?)",
#     row.Continent, row.Location, row.IsoCode, row.Population, row.PopulationDensity, row.MedianAge, row.GdpPerCapita, row.HumanDevelopmentIndex, row.LifeExpectancy, row.Aged65Older, row.Aged70Older, row.Alpha2)

# conn.commit()
# cursor.close()